In [ ]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
import time
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from openai.types.beta.threads import TextContentBlock

load_dotenv()

data_path = "../../data/air_passengers.csv"
prompt_path = "prompt/instruction.jinja2"

In [ ]:
assistant_name = "Code Interpreter"
question = "What is the average value of the target column"
instruction = (
    Environment(loader=FileSystemLoader(".")).get_template(prompt_path).render()
)

# get the client object
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",  # only support this version
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
print(client.beta.assistants.list().data)

In [ ]:
# delete assistant
# print (client.beta.assistants.list().data)
client.beta.assistants.delete(assistant_id="asst_m7CrUbLZilA4BJPRoP0DeyxD")

In [ ]:
# upload the data
data = client.files.create(file=open(data_path, "rb"), purpose="assistants")

# get the list of assistants
df_assistants = pd.DataFrame.from_records(
    [s.to_dict() for s in client.beta.assistants.list().data]
)
if not df_assistants.empty:
    df_assistants = df_assistants[df_assistants["name"] == assistant_name]

if df_assistants.shape[0] == 0:
    # create an assistant
    assistant = client.beta.assistants.create(
        name=assistant_name,
        instructions=instruction,
        tools=[{"type": "code_interpreter"}],
        tool_resources={"code_interpreter": {"file_ids": [data.id]}},
        model="gpt-4o",
        temperature=1,
        top_p=1,
    )
else:
    if df_assistants.shape[0] > 1:
        print("More than one assistant with the same name. Select the first one.")
    assistant_id = df_assistants.iloc[0]["id"]
    assistant = client.beta.assistants.retrieve(assistant_id=assistant_id)

# create a and run a thread
run = client.beta.threads.create_and_run(
    assistant_id=assistant.id,
    thread={"messages": [{"role": "user", "content": question}]},
)

# looping until the run completes or fails
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=run.thread_id, run_id=run.id)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=run.thread_id)

    # format the output
    for message in messages.data[::-1]:
        for item in message.content:
            if isinstance(item, TextContentBlock):
                print(f"{message.role}: {item.text.value}")

    # download the code


elif run.status == "requires_action":
    # the assistant requires calling some functions
    # and submit the tool outputs back to the run
    pass
else:
    print(run.status)